# Winery classification with the multivariate Gaussian

In this notebook, we return to winery classification, using the full set of 13 features.

## 1. Load in the data 

As usual, we start by loading in the Wine data set. Make sure the file `wine.data.txt` is in the same directory as this notebook.

Recall that there are 178 data points, each with 13 features and a label (1,2,3). As before, we will divide this into a training set of 130 points and a test set of 48 points.

In [1]:
# Standard includes
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal 

In [2]:
# Load data set.
data = np.loadtxt('wine.data.txt', delimiter=',')
# Names of features
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols', 
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                'OD280/OD315 of diluted wines', 'Proline']
# Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
np.random.seed(0)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

## 2. Fit a Gaussian generative model

We now define a function that fits a Gaussian generative model to the data.
For each class (`j=1,2,3`), we have:
* `pi[j]`: the class weight
* `mu[j,:]`: the mean, a 13-dimensional vector
* `sigma[j,:,:]`: the 13x13 covariance matrix

This means that `pi` is a 4x1 array (Python arrays are indexed starting at zero, and we aren't using `j=0`), `mu` is a 4x13 array and `sigma` is a 4x13x13 array.

In [3]:
len(set(trainy))

3

In [4]:
(trainy==3)

array([False,  True, False, False, False, False, False,  True, False,
       False,  True,  True, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
        True, False, False,  True, False, False, False, False, False,
       False,  True, False,  True, False,  True,  True, False,  True,
       False, False,  True,  True, False,  True,  True, False, False,
        True, False, False,  True, False, False, False, False, False,
       False, False, False, False,  True,  True, False, False, False,
        True,  True, False,  True, False, False, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False,  True, False,  True,  True, False, False, False,
        True, False, False, False, False, False, False, False,  True,
       False, False,

In [10]:
trainy[:100]

array([1., 3., 2., 1., 2., 2., 1., 3., 2., 2., 3., 3., 1., 2., 3., 2., 1.,
       1., 2., 1., 2., 1., 1., 2., 2., 2., 2., 2., 2., 3., 1., 1., 2., 1.,
       1., 1., 3., 2., 2., 3., 1., 1., 2., 2., 2., 1., 3., 2., 3., 1., 3.,
       3., 1., 3., 1., 2., 3., 3., 2., 3., 3., 1., 2., 3., 2., 2., 3., 2.,
       1., 2., 2., 2., 1., 1., 2., 2., 3., 3., 2., 2., 2., 3., 3., 1., 3.,
       2., 2., 2., 2., 2., 1., 1., 2., 1., 3., 1., 3., 1., 1., 2.])

In [17]:
for label in range(1,k+1):
    print(label)

1
2
3


In [18]:
set(trainy)

{1.0, 2.0, 3.0}

In [24]:
x[indices,:].shape

(33, 13)

In [71]:
indices = (y == 3)

array([1.31596970e+01, 3.37727273e+00, 2.40090909e+00, 2.08939394e+01,
       9.90303030e+01, 1.67606061e+00, 7.57272727e-01, 4.59696970e-01,
       1.20848485e+00, 7.38242421e+00, 6.86666667e-01, 1.67484848e+00,
       6.35909091e+02])

In [33]:
mu[label] = np.mean(x[indices,:],axis=0)

array([1.31596970e+01, 3.37727273e+00, 2.40090909e+00, 2.08939394e+01,
       9.90303030e+01, 1.67606061e+00, 7.57272727e-01, 4.59696970e-01,
       1.20848485e+00, 7.38242421e+00, 6.86666667e-01, 1.67484848e+00,
       6.35909091e+02])

In [77]:
mu.shape

(3, 13)

In [80]:
c = 1000
x = trainx
y = trainy


k = len(set(trainy)) #number of labels in y

d = (x.shape)[1]  # number of features
mu = np.zeros((k,d))
sigma = np.zeros((k,d,d))
pi = np.zeros(k)
for label in range(1,k+1):
    indices = (y == label)

    mu[label-1] = np.mean(x[indices,:],axis=0)
    sigma[label-1] = np.cov(x[indices,:], rowvar=0, bias=1)

    sigma[label-1] = sigma[label-1] + np.identity(d) * c #treating singularity

    pi[label-1] = float(sum(indices))/float(len(y))

In [81]:
def fit_generative_model(x,y,c=10000000):
    #k = 3  # labels 1,2,...,k
    k = len(set(trainy)) #number of labels in y
    
    d = (x.shape)[1]  # number of features
    mu = np.zeros((k,d))
    sigma = np.zeros((k,d,d))
    pi = np.zeros(k)
    for label in range(1,k+1):
        indices = (y == label)
        mu[label-1] = np.mean(x[indices,:], axis=0)
        sigma[label-1] = np.cov(x[indices,:], rowvar=0, bias=1)
        
        sigma[label-1] = sigma[label-1] + np.identity(d) * c #treating singularity

        pi[label-1] = float(sum(indices))/float(len(y))
    return mu, sigma, pi

In [82]:
# Fit a Gaussian generative model to the training data
mu, sigma, pi = fit_generative_model(trainx,trainy)

## 3. Use the model to make predictions on the test set

<font color="magenta">**For you to do**</font>: Define a general purpose testing routine that takes as input:
* the arrays `pi`, `mu`, `sigma` defining the generative model, as above
* the test set (points `tx` and labels `ty`)
* a list of features `features` (chosen from 0-12)

It should return the number of mistakes made by the generative model on the test data, *when restricted to the specified features*. For instance, using the just three features 2 (`'Ash'`), 4 (`'Magnesium'`) and 6 (`'Flavanoids'`) results in 7 mistakes (out of 48 test points), so 

        `test_model(mu, sigma, pi, [2,4,6], testx, testy)` 

should print 7/48.

**Hint:** The way you restrict attention to a subset of features is by choosing the corresponding coordinates of the full 13-dimensional mean and the appropriate submatrix of the full 13x13 covariance matrix.

In [86]:
multivariate_normal(mu[0],sigma[0])

In [89]:
k = len(set(trainy))-1
score = np.zeros((len(trainy),k))

for label in range(0,k):
    rv = multivariate_normal(mean=mu[label], cov=sigma[label])
    for i in range(0,len(trainy)):
        score[i,label] = np.log(pi[label]) + rv.logpdf(trainx[i,:])


In [109]:
# Compute log Pr(label|image) for each [test image,label] pair.
k = len(set(trainy))


score = np.zeros((k,len(trainy)))
for label in range(0,k):
    rv = multivariate_normal(mean=mu[label-1], cov=sigma[label-1])
    for i in range(0,k):
        score[i,label-1] = np.log(pi[label-1]) + rv.logpdf(testx[i,:])
predictions = np.argmax(score, axis=1)
# Finally, tally up score
errors = np.sum(predictions != testy)
print("Your model makes " + str(errors) + " errors out of 10000")

Your model makes 1 errors out of 10000


C:\Users\mnaum\Anaconda3\envs\Diesel\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  if sys.path[0] == '':


In [110]:
predictions

array([2, 2, 2], dtype=int64)

In [106]:
48*3

144

In [102]:
np.equal(predictions, testy)

ValueError: operands could not be broadcast together with shapes (130,) (48,) 

In [98]:
testy

array([1., 2., 3., 3., 1., 1., 3., 1., 3., 2., 1., 1., 2., 1., 3., 2., 3.,
       1., 3., 3., 3., 1., 2., 2., 2., 2., 3., 3., 2., 2., 1., 2., 3., 3.,
       1., 1., 3., 2., 2., 2., 1., 1., 1., 2., 2., 2., 1., 3.])

In [5]:
# Now test the performance of a predictor based on a subset of features
def test_model(mu, sigma, pi, features, tx, ty):
    ###
    ### Your code goes here
    ###

SyntaxError: unexpected EOF while parsing (<ipython-input-5-ceb7b9d5bca7>, line 5)

### <font color="magenta">Fast exercises</font>

*Note down the answers to these questions. You will need to enter them as part of this week's assignment.*

Exercise 1. How many errors are made on the test set when using the single feature 'Ash'?

In [ ]:
test_model(mu, sigma, pi, [2], testx, testy)

Exercise 2. How many errors when using 'Alcohol' and 'Ash'?

In [ ]:
test_model(mu, sigma, pi, [0,2], testx, testy)

Exercise 3. How many errors when using 'Alcohol', 'Ash', and 'Flavanoids'?

In [ ]:
test_model(mu, sigma, pi, [0,2,6], testx, testy)

Exercise 4. How many errors when using all 13 features?

In [ ]:
test_model(mu, sigma, pi, range(0,13), testx, testy)

Exercise 5. In lecture, we got somewhat different answers to these questions. Why do you think that might be?